# Boundary Element Method

In [1]:
import numpy as np
import sympy as sp
import math as mt
import scipy.integrate as integrate
from tabulate import tabulate

<b>Boundaries</b>

In [2]:
def x11(t: float):
    return sp.cos(t) / 2

def x12(t: float):
    return sp.sin(t) / 2

def x21(t: float):
    return sp.cos(t)

def x22(t: float):
    return sp.sin(t)

<b>Derivatives</b>

In [3]:
t = sp.Symbol('t')
j = sp.Symbol('j')

In [4]:
def evalf_d(expr, tau: float, j: int):
    xj = expr.evalf(subs = {t: j * h})
    return xj + tau * (expr.evalf(subs = {t: (j + 1) * h}) - xj) 

In [5]:
def d_xj(x, tau: float, j: int):
    expr = sp.diff(x(t))
    return evalf_d(expr, tau, j)

In [6]:
def d2_xj(x, tau: float, j: int):
    expr = sp.diff(sp.diff(x(t)))
    return evalf_d(expr, tau, j)

In [7]:
def d_x1(t: float, j: int):
    return [d_xj(x11, t, j), d_xj(x12, t, j)]

def d_x2(t: float, j: int):
    return [d_xj(x21, t, j), d_xj(x22, t, j)]

<b>Integrals</b>

In [19]:
def quad(K):
    return integrate.quad(K, 0, 1)[0] / (2 * np.pi)

<b>Normals</b>

In [9]:
def v1(t: float, j: int):
    z = sp.sqrt(np.dot(x1(t, j), x1(t, j)))
    return [xj(x12, t, j) / z, - xj(x11, t, j) / z]

def v2(t: float, j: int):
    z = sp.sqrt(np.dot(x2(t, j), x2(t, j)))
    return [xj(x22, t, j) / z, - xj(x21, t, j) / z]

<b>Elements Count</b>

In [10]:
N = 4 # count
h = 2 * sp.pi / N # step

<b>Aproximation with Linear Function</b>

In [11]:
def xj(x, t: float, j: int):
    xj = x(j * h) + t * (x((j + 1) * h) - x(j * h))
    return xj

def x1(t: float, j: int):
    return [xj(x11, t, j), xj(x12, t, j)]

def x2(t: float, j: int):
    return [xj(x21, t, j), xj(x22, t, j)]

<b>Kernels</b>

In [12]:
def K11(t: float, tau: float, j: int, k: int):
    if t != tau:
        sub = np.subtract(x1(t, k), x1(tau, j))
        return ((xj(x11, t, k) - xj(x11, tau, j)) * d_xj(x12, tau, j) - (xj(x12, t, k) - xj(x12, tau, j)) * d_xj(x11, tau, j)) / np.dot(sub, sub)
    else:
        return (d_xj(x11, t, k) * d2_xj(x12, t, k) - d_xj(x12, t, k) * d2_xj(x11, t, k)) / np.dot(d_x1(t, k), d_x1(t, k))
    
def K12(t: float, tau: float, j: int, k: int):
    sub = np.subtract(x1(t, k), x2(tau, j))
    return mt.log(1 / mt.sqrt(np.dot(sub, sub)))

def K21(t: float, tau: float, j: int, k: int):
    sub = np.subtract(x2(t, k), x1(tau, j))
    return np.dot(v2(t, k), v1(tau, j)) / np.dot(sub, sub) - 2 * np.dot(sub, v2(t, k)) * np.dot(sub, v1(tau, j)) / np.dot(sub,sub) ** 2

def K22(t: float, tau: float, j: int, k: int):
    if t != tau:
        sub = np.subtract(x2(t, k), x2(tau, j))
        return ((xj(x21, t, k) - xj(x21, tau, j)) * d_xj(x22, tau, j) - (xj(x22, t, k) - xj(x22, tau, j)) * d_xj(x21, tau, j)) / np.dot(sub, sub)
    else:
        return - (d_xj(x21, t, k) * d2_xj(x22, t, k) - d_xj(x22, t, k) * d2_xj(x21, t, k)) / np.dot(d_x2(t, k), d_x2(t, k))

In [13]:
def matrix_A():
    matrix = np.zeros((2 * N, 2 * N))
    
    for k in range(0, N):
        
        print("Перший квадрат")
        for j in range(0, N):
            matrix[k][j] = quad(lambda tau: K11(np.pi, tau, j, k) * mt.sqrt(np.dot(x1(tau, j), x1(tau, j))))
            if k == j:
                matrix[k][j] += 1 / 2
        
        j = 0
        print("Другий  квадрат")
        for i in range(N, 2 * N):
            matrix[k][i] = quad(lambda tau: K12(np.pi, tau, j, k) * mt.sqrt(np.dot(x2(tau, j), x2(tau, j))))
            j += 1

    k = 0
    for q in range(N, 2 * N):
        
        print("Третій квадрат")
        for j in range(0, N):
            matrix[q][j] = quad(lambda tau: K21(np.pi, tau, j, k) * mt.sqrt(np.dot(x1(tau, j), x1(tau, j))))
            
        print("Четвертий квадрат")
        j = 0
        for i in range(N, 2 * N):
            matrix[q][i] = quad(lambda tau: K22(np.pi, tau, j, k) * mt.sqrt(np.dot(x2(tau, j), x2(tau, j))))
            if q == i:
                matrix[q][i] += 1 / 2 
            j += 1
        k += 1
            
    print(matrix)
    return matrix

In [14]:
def vector_b():
    vec_b = np.zeros((2 * N, 1))
    
    for i in range(0, N):
        vec_b[i][0] = 1
        
    for i in range(N, 2 * N):
        vec_b[i][0] = 0
        
    print(vec_b)
    return vec_b

In [15]:
def solve_system(A, b):
    m = np.linalg.solve(A, b)
    print(m)
    
    u = 0

    for i in range(0, int(len(m) / 2)):
        u += m[i][0]
        
    for i in range(int(len(m) / 2), len(m)):
        u += m[i][0]

    return u

In [20]:
A = matrix_A()

Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Перший квадрат
Другий  квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
Третій квадрат
Четвертий квадрат
[[ 5.00866406e-01  1.30164172e-02 -3.60957381e-03 -1.04925916e-02
  -7.98293683e-02 -3.13196959e-02 -9.74913853e-02 -1.25035017e-01]
 [-1.04925916e-02  5.00866406e-01  1.30164172e-02 -3.60957381e-03
  -1.25035017e-01 -7.98293683e-02 -3.13196959e-02 -9.74913853e-02]
 [-3.60957381e-03 -1.04925916e-02  5.00866406e-01  1.30164172e-02
  -9.74913853e-02 -1.25035017e-01 -7.98293683e-02 -3.13196959e-02]
 [ 1.30164172e-02 -3.60957381e-03 -1.04925916e-02  5.00866406e-01
  -3.13196959e-02 -9.74913853e-02 -1.25035017e-01 -7.98293683e-02]
 [ 2.65869114e-04  4.26522357e-03 -1.17883677e-03 -3.38161481e-03
   5.01732813e-01  2.60328344e-02 -7.21914762e-03 -2.09851832e-02]
 [-3.38161481e-03  2.65869114e-04  4.26522357e-03 -1.17883677e-03
  -2.09851832

In [21]:
print(tabulate(A))
print("det = " + str(np.linalg.det(A)))

------------  ------------  ------------  ------------  -----------  -----------  -----------  -----------
 0.500866      0.0130164    -0.00360957   -0.0104926    -0.0798294   -0.0313197   -0.0974914   -0.125035
-0.0104926     0.500866      0.0130164    -0.00360957   -0.125035    -0.0798294   -0.0313197   -0.0974914
-0.00360957   -0.0104926     0.500866      0.0130164    -0.0974914   -0.125035    -0.0798294   -0.0313197
 0.0130164    -0.00360957   -0.0104926     0.500866     -0.0313197   -0.0974914   -0.125035    -0.0798294
 0.000265869   0.00426522   -0.00117884   -0.00338161    0.501733     0.0260328   -0.00721915  -0.0209852
-0.00338161    0.000265869   0.00426522   -0.00117884   -0.0209852    0.501733     0.0260328   -0.00721915
-0.00117884   -0.00338161    0.000265869   0.00426522   -0.00721915  -0.0209852    0.501733     0.0260328
 0.00426522   -0.00117884   -0.00338161    0.000265869   0.0260328   -0.00721915  -0.0209852    0.501733
------------  ------------  ------------  ----

In [22]:
b = vector_b()
print(solve_system(A, b))

[[1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]]
[[2.00095626e+00]
 [2.00095626e+00]
 [2.00095626e+00]
 [2.00095626e+00]
 [1.17594936e-04]
 [1.17594936e-04]
 [1.17594936e-04]
 [1.17594936e-04]]
8.004295434075674
